In [1]:
spark.conf.set("spark.sql.shuffle.partiotins",16)

In [2]:
val df = spark.read.format("json")
  .load("/home/jovyan/work/Spark-The-Definitive-Guide/data/flight-data/json/2015-summary.json")

df = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]


[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]

In [3]:
df.schema

StructType(StructField(DEST_COUNTRY_NAME,StringType,true), StructField(ORIGIN_COUNTRY_NAME,StringType,true), StructField(count,LongType,true))

In [4]:
df.sort("count").show(2)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|          Moldova|      United States|    1|
|    United States|            Croatia|    1|
+-----------------+-------------------+-----+
only showing top 2 rows



df.orderBy("count","DEST_COUNTRY_NAME").show(2)

In [6]:
df.limit(1).show()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
+-----------------+-------------------+-----+



In [8]:
import org.apache.spark.sql.functions.{expr}
df.orderBy(expr("count desc")).limit(2).show()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|          Singapore|    1|
|          Moldova|      United States|    1|
+-----------------+-------------------+-----+



#### Repartion and Coalesce

In [9]:
df.rdd.getNumPartitions

1

In [10]:
df.repartition(5)

[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]

Partition based on columns

In [11]:
import org.apache.spark.sql.functions.{col}

In [12]:
df.repartition(col("DEST_COUNTRY_NAME"))

[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]

In [14]:
df.rdd.getNumPartitions

1

In [16]:
df.repartition(5,col("DEST_COUNTRY_NAME"))
df.rdd.getNumPartitions

1

In [17]:
df.repartition(5,col("DEST_COUNTRY_NAME")).coalesce(2)

[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]

In [19]:
val collectDF = df.limit(10)

collectDF = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]


[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]

In [20]:
collectDF.take(5)
collectDF.show()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
|    United States|          Singapore|    1|
|    United States|            Grenada|   62|
|       Costa Rica|      United States|  588|
|          Senegal|      United States|   40|
|          Moldova|      United States|    1|
+-----------------+-------------------+-----+



In [22]:
collectDF.show(5,false)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|United States    |Romania            |15   |
|United States    |Croatia            |1    |
|United States    |Ireland            |344  |
|Egypt            |United States      |15   |
|United States    |India              |62   |
+-----------------+-------------------+-----+
only showing top 5 rows



In [23]:
collectDF.collect()

Array([United States,Romania,15], [United States,Croatia,1], [United States,Ireland,344], [Egypt,United States,15], [United States,India,62], [United States,Singapore,1], [United States,Grenada,62], [Costa Rica,United States,588], [Senegal,United States,40], [Moldova,United States,1])

In [30]:
for (i <- collectDF.collect())
{
    println(i(2).toString()+" "+i(0))
}

15 United States
1 United States
344 United States
15 Egypt
62 United States
1 United States
62 United States
588 Costa Rica
40 Senegal
1 Moldova
